In [1]:
import sys
sys.path.append('C:\Anaconda3\Lib\site-packages')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm, tqdm_notebook

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

### Optimize kNN and re-do predictions
Implement grid search with custom MAP@3 scoring function.

In [2]:
df = pd.read_csv('train.csv')

In [ ]:
y_hash = {y: i for i, y in enumerate(np.unique(df.place_id))}
df['place_hash'] = df['place_id'].map(y_hash)
df.head()

In [ ]:
def my_scorer(clf, X, y_true):
    ''' Custom scoring function for sklearn.grid_search.GridSearchCV
        that calcualtes the MAP@3 scoring metric as defined by Kaggle
        for this competition. See evaluation page for more details
        on the metric: https://www.kaggle.com/c/facebook-v-predicting-check-ins/details/evaluation
        
        clf - Estimator with grid search specified parameters and
              fit with grid search cross valdiation data.
        X - Features with shape [n_features, n_dimensions].
        y - True labels to test classification accuracy with
            shape [n_features]. When fitting grid search must
            use y.values for this function to work properly.
            e.g. gs.fit(X_train, y_train.values) '''
    print(X.shape)
    print(y_true.shape)
    
    y_pred_proba = clf.predict_proba(X)
    class_labels = clf.classes_

    # Sort the classifications based on highest probability,
    # making sure to keep track of class labels properly
    p_pairs = [sorted([(c, p) for c, p in zip(class_labels, y_pred_proba[j])],
                      key=lambda x: x[1], reverse=True)
                for j in range(y_pred_proba.shape[0])]
    
    # Get the top 3 predictions by selecting the class
    # label piece of the tuple (element 0) for each sample
    y_top_3 = [[p[i][0] for i in range(3)]
               for p in p_pairs]
    
    # Calculate the MAP@3 score,
    # where the sum over P(k) is equal to
    # 1 if correct prediction is 1st
    # 1/2 if correct prediction is 2nd
    # 1/3 if correct prediction is 3rd
    # 0 otherwise
    MAP_score = [[(y == y_true[j])/(i+1) for i, y in enumerate(y_sample)]
                 for j, y_sample in enumerate(y_top_3)]
    MAP_score = np.sum(MAP_score, axis=1).mean()
    
    return MAP_score

For testing, will use a small slice of the dataframe for speed reasons.

In [ ]:
df2 = df.iloc[0:1000, :].copy()

# # Find features that only occur once and remve them
# from collections import Counter
# place_counts = Counter(df2.place_id.values)
# df2['occurrences'] = df2['place_id'].map(place_counts)
# df2 = df2[(df2.occurrences > 1)]
# len(df2)

In [ ]:
features = ['x', 'y']
X_train, X_test, y_train, y_test = train_test_split(
    df2[features], df2['place_id'],
    test_size=0.3, random_state=1)

std_scaler = StandardScaler()
std_scaler.fit(X_train)
X_train_std = std_scaler.transform(X_train)
X_test_std = std_scaler.transform(X_test)

In [ ]:
from collections import Counter
place_counts = Counter(df.place_id.values)

In [ ]:
sorted([(id, count) for id, count in place_counts.items()],
        key=lambda x: x[1], reverse=True)[:3]

In [ ]:
gs = GridSearchCV(estimator=KNeighborsClassifier(weights='distance'),
                  param_grid=[{'n_neighbors': [5, 6]}],
                  cv=2,
                  scoring=my_scorer)

gs.fit(X_train_std, y_train.values)

In [ ]:
gs.grid_scores_

In [ ]:
len(df_test)

### Make predictions for test set
Load test sampes into dataframe.

In [3]:
df_test = pd.read_csv('test.csv')
df_test.head()

,row_id,x,y,accuracy,time
0,0,0.1675,1.3608,107,930883
1,1,7.3909,2.5301,35,893017
2,2,8.0978,2.3473,62,976933
3,3,0.9990,1.0591,62,907285
4,4,0.6670,9.7254,40,914399


In [4]:
features = ['x', 'y']
X_train = df[features]
y_train = df.place_id.values

std_scaler = StandardScaler().fit(X_train)
X_train_std = std_scaler.transform(X_train)

Train estimator.

In [5]:
%%time
clf = KNeighborsClassifier(n_neighbors=10, weights='distance')
clf.fit(X_train_std, y_train)
# gs = GridSearchCV(estimator=clf,
#                   param_grid=[{'n_neighbors': [15],
#                                'weights': ['distance']}],
#                   cv=3,
#                   scoring=my_scorer,
#                   n_jobs=-1)
# gs.fit(X_train_std, y_train)
# clf = gs.best_estimator_

Wall time: 1min 46s


In [6]:
features = ['x', 'y']
X_test = df_test[features]
X_test_std = std_scaler.transform(X_test)

y_pred = clf.predict(X_test_std)

In [7]:
df_test['p1'] = y_pred

In [8]:
def write_dataframe(df):
    df_ = df.p1#.str.cat([df.p2, df.p3], sep=' ')
    df_.name = 'place_id'
    df_.to_csv('submission_2.csv', index=True,
                header=True, index_label='row_id')

In [9]:
df_write = df_test[['p1']]
write_dataframe(df_write.astype(str))

Try and write three unique predictions for each target using predict_proba (might not finish in time). Use a new model for more variety (can mix them together to try and achieve higher score).

In [15]:
clf = KNeighborsClassifier(n_neighbors=15)
clf.fit(X_train_std, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=15, p=2,
           weights='uniform')

In [18]:
X_test_std[:,1].shape

(8607230,)

In [21]:
def write_predictions(file, clf, x1, x2):
    with open(file, 'w') as f:
        f.write('row_id,place_id\n')
        for r_id, x, y in tqdm_notebook(zip(df.row_id.values,
                              x1,
                              x2)):
            p_id = sorted([(clf.classes_[i], p) for i, p in enumerate(clf.predict_proba([[x, y]])[0])],
                          key=lambda x: x[1], reverse=True)[:3]
            f.write('{0:d},{1:d} {2:d} {3:d}\n'.format(r_id,
                                                       p_id[0][0],
                                                       p_id[1][0],
                                                       p_id[2][0]))

In [ ]:
%%time
write_predictions('submission_3.csv', clf, X_test_std[:,0], X_test_std[:,1])

Mini test-zone.

In [ ]:
y_new.shape

In [ ]:
i_last = 0
i = 0
X_test_std[i_last:(i+1)*batch_size]
y_new = np.array([[6131996960, 3831655216, 4393146716],
                    [4505710601, 4223683383, 2411939657]])
y_pred_3 = np.empty(shape=(0, 3))
np.concatenate((y_pred_3, y_new), axis=0)

In [ ]:
class_labels[y_pred_proba[0] != 0]
y_pred_proba[0][y_pred_proba[0] != 0]

Make predictions for ~8 million samples. This will crash from memory overflow every time. Better to use predict_proba on each sample and write to file as implemented in my other notebook but that takes too long.

In [ ]:
features = ['x', 'y']
X_test = df_test[features]
X_test_std = std_scaler.transform(X_test)

import time

def online_predict(y_pred_3, i, batch_size, i_last):
    ''' Online prediction loop. '''

#     start = time.time()
    i_new = (i+1)*batch_size
    try:
        y_pred_proba = clf.predict_proba(X_test_std[i_last:i_new, :])
    except:
        y_pred_proba = clf.predict_proba(X_test_std[i_last:, :])
#     end = time.time()
#     print((end - start)/60)
    
#     start = time.time()
    p_pairs = [sorted([(c, p) for c, p in zip(clf.classes_[y_pred_proba[j] != 0],
                                              y_pred_proba[j][y_pred_proba[j] != 0])],
                      key=lambda x: x[1], reverse=True)
                for j in range(y_pred_proba.shape[0])]
#     end = time.time()
#     print((end - start)/60)

#     start = time.time()
    # Get the top 3 predictions by selecting the class
    # label piece of the tuple (element 0) for each sample
    y_top_3 = np.array([[p[i][0] for i in range(min(len(p), 3))]+[0]*(3-min(len(p), 3))
                       for p in p_pairs])
    print(y_top_3.shape)
    print(y_pred_3.shape)
    print(y_top_3)
    y_pred_3 = np.concatenate((y_pred_3, y_top_3), axis=0)
#     end = time.time()
#     print((end - start)/60)
    
    i_last = i
    return y_pred_3, i_last

batch_size = 10000
i_last = 0
y_pred_3 = np.empty(shape=(0,3))
# y_pred_3 = np.array([[]])
for i in tqdm(range(int(np.ceil(len(X_test_std)/batch_size)))):
    y_pred_3, i_last = online_predict(y_pred_3, i, batch_size, i_last)
    
df_test['p1'] = y_pred_3.T[0]
df_test['p2'] = y_pred_3.T[1]
df_test['p3'] = y_pred_3.T[2]

# # Calculate the MAP@3 score,
# # where the sum over P(k) is equal to
# # 1 if correct prediction is 1st
# # 1/2 if correct prediction is 2nd
# # 1/3 if correct prediction is 3rd
# MAP_score = [[(y == y_true[j])/(i+1) for i, y in enumerate(y_sample)]
#              for j, y_sample in enumerate(y_top_3)]

In [ ]:
def write_dataframe(df):
    df_ = df.p1.str.cat([df.p2, df.p3], sep=' ')
    df_.name = 'place_id'
    df_.to_csv('submission_2.csv', index=True,
                header=True, index_label='row_id')

In [ ]:
df_write = df_test[['p1', 'p2', 'p3']]
write_dataframe(df_write.astype(str))

In [ ]:
[1., 2.]+[3.]*2

In [ ]:
a=3

In [ ]:
pp = [[(61883, 0.21533447911348993), (34258, 0.1539756877723768), (41072, 0.15343296963015976), (37376, 0.10552160063432142), (20081, 0.093207528705804343), (93066, 0.067748229481564046), (84435, 0.05517781092934726), (5735, 0.045790499409853837), (79949, 0.038691927954741592), (12033, 0.035696002229168453), (64871, 0.035423264139172586)], [(42476, 0.19014899426178219), (39032, 0.16564611628208384), (17285, 0.10287561875236013), (62020, 0.084593176732650169), (29810, 0.083089221404869157), (17936, 0.064802976416084707), (57964, 0.060227628508823156), (23374, 0.059717682514312732), (44018, 0.051966259312535359), (52446, 0.051743080284601034), (34719, 0.043251620090330491), (15448, 0.041937625439566972)], [(18557, 0.43423521843349816), (82540, 0.38347117524948388), (1388, 0.087249547055486029), (38402, 0.053951774633285224), (75769, 0.041092284628246656)], [(84159, 0.65165115791449058), (27211, 0.30166186765686642), (27327, 0.046686974428643016)], [(45595, 0.7171673602450791), (66669, 0.16681670490127157), (87635, 0.11601593485364922)], [(88809, 0.60814322152558264), (105054, 0.3918567784744173)], [(78864, 0.64575147337487937), (26288, 0.17168922954354002), (75514, 0.052095662851282476), (91046, 0.048489857560822573), (70562, 0.046081455980905975), (58951, 0.035892320688569505)], [(18149, 0.31465843610497268), (3197, 0.15038001185520206), (90912, 0.13214408525533641), (90209, 0.089619425025800281), (98393, 0.073389573447385878), (26381, 0.06762480810237842), (39759, 0.062000842625947163), (40493, 0.06031171897362831), (68141, 0.049871098609348727)], [(65952, 0.31074996432106311), (65392, 0.10702650877177056), (56778, 0.1008131828543166), (63833, 0.10025841570328371), (44482, 0.093840268102816787), (41530, 0.08918579469324979), (65382, 0.053837747075445384), (29583, 0.049922949108990873), (44766, 0.048093667698521345), (99695, 0.046271501670541844)], [(71997, 0.51620469414988945), (28590, 0.15265379184205369), (75661, 0.086302060003291423), (35063, 0.080493054811579159), (74138, 0.074392284662816555), (7719, 0.046954993518305281), (12592, 0.042999121012064503)], [(9426, 0.60361916344290023), (86773, 0.25610145983701033), (49571, 0.10479145915620335), (31986, 0.035487917563886091)], [(45850, 0.18635241824654666), (37934, 0.18419636814908311), (64264, 0.17735263956633435), (48525, 0.089207421191089098), (14147, 0.074063112939646056), (71682, 0.066666819087156354), (102139, 0.064082179230509906), (9245, 0.055448898788023719), (11607, 0.054264224515096424), (63010, 0.048365918286514271)], [(96715, 0.20820933687316825), (76985, 0.096892942432124554), (24557, 0.094626505831629237), (36057, 0.094431621808255164), (103540, 0.080869468661822444), (102897, 0.060319563743636756), (57860, 0.0603107701415719), (36354, 0.058699211085796899), (63803, 0.051507960976709719), (53318, 0.05105436975069897), (34741, 0.049171976801326768), (43067, 0.047347854815366276), (42071, 0.046558417077893127)], [(52153, 0.53898148811169089), (17550, 0.22221307506896187), (29413, 0.096023032887396748), (107913, 0.072599747183298868), (18466, 0.070182656748651567)], [(76965, 0.48464199773505434), (56176, 0.30951693444829442), (33350, 0.20584106781665126)], [(89650, 0.53205557743862397), (41047, 0.38982115670746387), (17600, 0.043696799556172311), (107520, 0.034426466297739815)], [(84414, 0.63775385344402657), (25915, 0.32319245756145309), (101500, 0.039053688994520441)], [(64124, 0.30794955394662055), (93860, 0.20823042373063649), (43642, 0.1823591420025292), (26939, 0.15880234624280828), (91715, 0.059721896293736303), (42472, 0.041825576786267096), (49000, 0.041111060997402048)], [(22730, 0.30309094047686025), (44227, 0.21048885179607674), (33588, 0.12611652995481654), (30154, 0.070162950598749357), (99942, 0.067299613725412599), (102424, 0.048898615719520915), (8827, 0.048045488848686264), (26125, 0.04468996135369175), (9236, 0.04108081999862595), (13359, 0.040126227527559719)], [(9402, 0.44942204278160719), (86022, 0.36364958048610641), (20313, 0.064602145060066549), (42012, 0.032297243067403426), (45890, 0.032091601137966518), (8473, 0.029667014008411499), (31122, 0.028270373458438247)], [(49430, 0.2426944351511422), (34465, 0.13516969629728698), (89020, 0.12547021415611281), (75088, 0.12159172856697305), (106023, 0.077046383241345454), (104751, 0.068597820716718189), (41729, 0.063689308570356554), (15207, 0.05605345436860399), (6973, 0.055553492568730649), (48138, 0.054133466362730115)], [(72125, 0.44121008101377918), (15926, 0.19961118908536987), (39810, 0.096077537549656061), (59471, 0.086805567343300402), (69499, 0.074314614973480742), (36550, 0.057549168653118508), (107462, 0.04443184138129521)], [(82270, 0.50855062691387198), (93306, 0.17952719748965287), (46811, 0.092966291630446965), (56850, 0.083670907323248375), (19190, 0.072222987007940886), (21703, 0.063061989634839041)], [(5221, 0.3498745893993267), (106990, 0.34099255766636311), (103251, 0.18910295104198582), (66291, 0.060166769891842109), (76184, 0.059863132000482301)], [(100640, 0.32223582219988256), (13274, 0.16264527883178009), (98219, 0.11229836325117475), (74419, 0.078968678820613175), (6855, 0.070649351833170296), (42862, 0.067821266515737527), (76559, 0.05526847482551233), (24408, 0.053825952937952844), (79259, 0.0383995161404928), (25206, 0.037887294643683567)], [(78852, 0.63689131437698598), (72728, 0.12437994974007557), (99744, 0.11427373614128032), (17663, 0.067297462234425662), (46486, 0.057157537507232452)], [(58690, 0.20978490677862663), (46249, 0.11024463756110077), (47331, 0.10488289775438062), (64061, 0.082813822429865874), (61955, 0.079525122411366389), (91228, 0.071512138611911191), (26089, 0.068076655611330444), (27039, 0.062895133852536991), (37734, 0.05865752736593851), (83675, 0.052031394213517129), (61281, 0.05065000352181364), (78730, 0.048925759887611921)], [(96065, 0.74285057819198996), (447, 0.11189705597836788), (18224, 0.09136818617222732), (74780, 0.05388417965741487)], [(37199, 0.22261669322769487), (27349, 0.12000388438628164), (58174, 0.11326234255747197), (24557, 0.081513918742422045), (51698, 0.078860989468636358), (44487, 0.066845015643998348), (79876, 0.065103053056867352), (27564, 0.053292393049108253), (36354, 0.050668536937035491), (78141, 0.049734560827806799), (41210, 0.049372529913036353), (79943, 0.048726082189640453)], [(81436, 0.52674578648273684), (77390, 0.16249776426351084), (32319, 0.1237776045480433), (71643, 0.065052268557662066), (107009, 0.061544452702209512), (2842, 0.060382123445837259)], [(45730, 0.22017888008739106), (60797, 0.16991662431008309), (57088, 0.15293660615592439), (100613, 0.15175640628060552), (71715, 0.086999113314718379), (53044, 0.079867054829629913), (69966, 0.073349207806275685), (82609, 0.064996107215371898)], [(96106, 0.79577563374257021), (5668, 0.056053647041640689), (54665, 0.053834342113778015), (38759, 0.048323708260875738), (98154, 0.046012668841135397)], [(30765, 0.68588384379724887), (38486, 0.11443781941746499), (10290, 0.06618992117057132), (88114, 0.048841015121390045), (69492, 0.043964453817807329), (20255, 0.040682946675517445)], [(57639, 0.62844301296003002), (92524, 0.13554783423828065), (101137, 0.10112695393219866), (17309, 0.069361237607273959), (80386, 0.065520961262216765)], [(61005, 0.28559411839902177), (96036, 0.13443086207637259), (54186, 0.11886064227066757), (74383, 0.073504534502804003), (43226, 0.069337086754357791), (39783, 0.068320807904077241), (1097, 0.05827302069565099), (38096, 0.044850742560143438), (9453, 0.040134938595992181), (7908, 0.037640858786320563), (93491, 0.035507698496129468), (68227, 0.033544688958462347)], [(28563, 1.0)], [(6337, 1.0)], [(43264, 0.29618556665341483), (36983, 0.22036334563781007), (41948, 0.093410351488851934), (54516, 0.082264077760416221), (15013, 0.080923507073139958), (39164, 0.051284752703871991), (94565, 0.049443712831418817), (97940, 0.048623295671943748), (6315, 0.042405658383152911), (56854, 0.035095731795979392)], [(19197, 0.48998106818401588), (46871, 0.32830286243340745), (64953, 0.1817160693825767)], [(35108, 0.41675505465447238), (46136, 0.17184986596497642), (95207, 0.16645277037452297), (59854, 0.10818026050278749), (72942, 0.099410874724592493), (6801, 0.037351173778648238)], [(61929, 0.31716574864184327), (37108, 0.271497629040931), (99499, 0.095924728797601666), (60986, 0.073607026592975119), (106, 0.065783335383235081), (104488, 0.061006007268469914), (3765, 0.058867271623548163), (76970, 0.056148252651395739)], [(81369, 0.31553612222757743), (103796, 0.17487016661432789), (28621, 0.16386444824911833), (101895, 0.12830166619977498), (19333, 0.100105572315695), (23304, 0.059723200525549365), (105342, 0.05759882386795713)], [(13008, 0.22867448759289527), (94455, 0.19164104875761875), (14299, 0.16587019498462552), (50524, 0.14375935654096819), (55293, 0.13217599457100687), (94228, 0.050651835510155653), (108056, 0.045325994116028008), (105209, 0.041901087926701913)], [(81148, 0.39270610761147323), (97134, 0.14752689351337805), (104198, 0.12443821014775935), (2135, 0.10382428776132398), (94440, 0.093376721139617744), (96285, 0.078141049450208999), (81747, 0.059986730376238598)], [(10224, 0.27497313785990407), (57020, 0.12794478032768553), (78051, 0.11581342966280349), (107324, 0.096762428749145965), (66180, 0.089203329711284446), (45948, 0.063021906837094702), (88636, 0.05312945975108968), (81120, 0.047167964737072382), (35019, 0.045215661363996199), (66861, 0.043731258217776757), (93743, 0.043036642782147007)], [(21144, 0.71729263630244477), (42744, 0.20253639647332122), (10704, 0.041871772525828223), (67986, 0.038299194698405953)], [(61291, 0.35518224240939578), (5121, 0.23604014378543955), (28721, 0.15223570788556842), (88073, 0.1183447239626341), (62624, 0.10526720963874719), (59691, 0.032929972318214942)], [(89992, 0.3498946463749345), (46704, 0.23588145383817613), (90202, 0.20905862504927206), (50269, 0.11224037715036629), (52352, 0.048551177234787461), (28015, 0.044373720352463661)], [(3784, 0.25257288101175795), (62759, 0.23058962421049206), (47423, 0.15501855692330418), (7862, 0.11357190743368929), (50255, 0.11031069939192639), (3942, 0.062480802696001925), (57260, 0.043197925243533365), (90244, 0.03225760308929488)], [(52346, 0.37088054805803594), (52823, 0.15541323124222378), (27872, 0.1151543340708527), (101960, 0.087115633264276876), (5152, 0.073576957158133921), (69192, 0.044465536402252162), (28075, 0.041014250908690229), (21694, 0.037701875263633271), (52721, 0.03762918637499936), (67183, 0.03704844725690179)], [(84383, 0.35977472881678774), (98897, 0.21103527865038116), (72367, 0.17358295255992637), (84731, 0.084819707861179858), (103054, 0.064570923787840517), (24229, 0.054758803088308043), (30109, 0.051457605235576374)], [(8453, 0.51070753514273937), (41325, 0.32023571350237257), (90428, 0.032145346073023891), (26625, 0.028268131592123543), (35464, 0.028265997707210944), (94149, 0.02406012139485119), (42914, 0.020451923664455085), (60495, 0.018264633973443321), (82489, 0.017600596949779942)], [(25704, 0.68293119601513053), (20983, 0.13944977381758769), (78660, 0.11868143357110154), (16579, 0.058937596596180357)], [(60850, 0.72607336850647497), (14021, 0.13652361254479045), (20097, 0.070089543388723835), (86299, 0.033782683158072667), (41492, 0.033530792401937945)], [(62541, 0.36199633931662023), (38119, 0.15535293247930893), (37529, 0.15194309783436605), (11156, 0.12732713464841622), (96048, 0.092201101077568348), (8112, 0.074134135402295756), (42593, 0.037045259241424398)], [(81706, 0.64683514245184182), (75385, 0.18777994512953233), (63461, 0.10027493951174433), (54262, 0.065109972906881572)], [(86600, 0.31847471724190374), (30728, 0.20676652413342725), (66428, 0.10174742715710697), (49427, 0.077400160321830697), (48200, 0.075779496980320077), (43422, 0.062677163380901652), (107027, 0.05622298677055397), (38081, 0.051433559324369237), (72978, 0.049497964689586386)], [(70307, 0.5537940011081991), (4812, 0.17939052581065834), (85298, 0.070811337402235122), (22444, 0.057751007435494148), (795, 0.054500319621501377), (79199, 0.046069367199790309), (2104, 0.037683441422121579)], [(32096, 0.70912494751997857), (51180, 0.18539924195679625), (86049, 0.10547581052322512)], [(16445, 0.27229166592500459), (24414, 0.1903873724828557), (64392, 0.14493013364973573), (90379, 0.13918888077597238), (53438, 0.095964084500319291), (49565, 0.079828607555696637), (12623, 0.077409255110415756)], [(37551, 0.22355072635789253), (58056, 0.20334693651227129), (103707, 0.1707515552637883), (89410, 0.14161908875468346), (76463, 0.13573418274908641), (30690, 0.054770422454115709), (72299, 0.038088994648261758), (48225, 0.032138093259900684)], [(90926, 0.43063998685393146), (91784, 0.16330930011104938), (92389, 0.10878662362848847), (108372, 0.090742742216647479), (33458, 0.078246734557244166), (47779, 0.036836406487269178), (62336, 0.035765245667885914), (90157, 0.031001248682337054), (37296, 0.024671711795146833)], [(77569, 0.48131237132304783), (30480, 0.19746998561514631), (47509, 0.10661399171141415), (87340, 0.08181855643265537), (22870, 0.052390631023985834), (22068, 0.042957984909419031), (16655, 0.037436478984331625)], [(98887, 0.56795906804099927), (19002, 0.18437001895499949), (99469, 0.072143109958759402), (6030, 0.068318219238502259), (104393, 0.055913302096490994), (101264, 0.051296281710248505)], [(22462, 0.81658108075685054), (86621, 0.062851574750542288), (54190, 0.048070558563345869), (82260, 0.036709939684141726), (53564, 0.035786846245119491)], [(24850, 0.37932644548455213), (31711, 0.35548377287463351), (25487, 0.073683526998747459), (36995, 0.052694657203390012), (61369, 0.039640610244299913), (51266, 0.033809433809771187), (7179, 0.033159236267331624), (64408, 0.032202317117274087)], [(32786, 0.31150592887708095), (102792, 0.16800360738047948), (56277, 0.12526106040122895), (32133, 0.10138397637983979), (2980, 0.069684970270871852), (67052, 0.066204794300021297), (21809, 0.060518094278673197), (67240, 0.054319559707392946), (51381, 0.043118008404411566)]]

In [ ]:
np.concatenate(np.empty(shape=(0,3)), np.array([[1, 2, 3], [2, 3, 4]]))

In [ ]:
np.empty(shape=(1,0))

In [ ]:
np.array([np.array([1, 2, 3]), np.array([2, 3, 4])].shape